<b>Código principal para cargar datos.</b>
<p> Configurar fechas de carga, seriales de boyas y rutas para archivos en \Configs\configuracion_general.py </p>


In [ ]:
####################### N O  T O C A R ############################################
%load_ext autoreload
%autoreload 2
import sys
import os

# Agrega la ruta raíz del proyecto si no está
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# import Configs.configuracion_general as config
import Funciones.Carga.cargar_datos_csv as carga 
from Funciones.Utils.utilidades import *
from Funciones.Correctores.corrector_utils import *
from Funciones.Utils.utils_get_config_vars import * 
##################################################################################

### Paso 1. Cargar los datos de la sonda y de despliegue

In [ ]:
rutas_de_sondas, seriales_encontrados = carga.buscar_nombre_de_archivo_de_sonda()

In [ ]:
# rutas_de_sondas

In [ ]:
diccionario_de_datos_de_sondas = carga.cargar_datos_de_sonda(rutas_de_sondas, seriales_encontrados)

In [ ]:
# diccionario_de_datos_de_sondas["4878218"][0:40]

In [ ]:
df_excel_de_despliegue = carga.leer_excel_de_despliegue_de_sondas(seriales_encontrados)

In [ ]:
# df_excel_de_despliegue

### Paso 2. Seleccionar los datos dentro del rango de fechas del estudio, prepararlos y guardarlos

In [ ]:
diccionario_de_sondas_en_fechas = carga.seleccionar_rango_de_fechas(diccionario_de_datos_de_sondas, df_excel_de_despliegue, seriales_encontrados)
datos_de_sondas_sin_duplicados = carga.buscar_y_eliminar_duplicados(diccionario_de_sondas_en_fechas)
datos_ordenados = carga.ordernar_datos_por_fecha(datos_de_sondas_sin_duplicados)

In [ ]:
# diccionario_de_sondas_en_fechas["4878205"].head()

In [ ]:
datos_con_tspan_redondeado = carga.crear_tspan_redondeado(datos_ordenados) # diccionario con tspan redondeado
datos_sin_repetidos = carga.existen_fechas_redondeadas_duplicadas(datos_con_tspan_redondeado) # Verifica si existen fechas redondeadas duplicadas, si existen lanza un error

In [ ]:
# datos_sin_repetidos["4878503"] # Muestra de los datos de la primera sonda cualquiera

In [ ]:
# A continuación se crea el dataframe vacío con todas las fechas que pueden existir y se combina con el dataframe de cada sonda
columnas = carga.get_cols_names_sin_tspan(diccionario = datos_sin_repetidos)
diccionario_con_dfs_vacios = carga.crear_diccionario_con_dataframes_vacios(diccionario= datos_sin_repetidos, keys=columnas)
datos_finales = carga.merge_df_vacio_con_datos(datos_sin_repetidos, diccionario_con_dfs_vacios) #datos finales realmente son los datos del periodo de vigencia del estudio

In [ ]:
# datos_finales["4878205"].head(10) # Ejemplo de los datos de salida

In [ ]:
# Eliminar datos espurios (solo se revisa si hay valores de rapidez superiores a 2 m/s y se elimina toda la fila)
datos_finales = eliminar_datos_espurios(datos_finales)

In [ ]:
# Agregar componentes de la velocidad al diccionario con los dataframe de cada sonda
datos_finales = carga.agregar_componentes_de_la_velocidad(datos_finales)

In [ ]:
# datos_finales["4878503"]

### Paso 3. Seleccionar todos los datos antes de la fecha de estudio

In [ ]:
# En este caso estoy buscando las fechas desde la instalación hasta la fecha de inicio del análsisis
diccionario_de_sondas_en_fechas_anteriores = carga.seleccionar_rango_de_fechas(diccionario = diccionario_de_datos_de_sondas, 
                                                                    df_excel_de_despliegue =df_excel_de_despliegue, 
                                                                    seriales_encontrados = seriales_encontrados,
                                                                    buscar_fechas_anteriores_al_estudio = True)
datos_de_sondas_sin_duplicados_anteriores = carga.buscar_y_eliminar_duplicados(diccionario_de_sondas_en_fechas_anteriores)
datos_ordenados_anteriores = carga.ordernar_datos_por_fecha(datos_de_sondas_sin_duplicados_anteriores)
datos_con_tspan_redondeado_anteriores = carga.crear_tspan_redondeado(datos_ordenados_anteriores) # diccionario con tspan redondeado
datos_sin_repetidos_anteriores = carga.existen_fechas_redondeadas_duplicadas(datos_con_tspan_redondeado_anteriores) # Verifica si existen fechas redondeadas duplicadas, si existen lanza un error
# A continuación se crea el dataframe vacío con todas las fechas que pueden existir y se combina con el dataframe de cada sonda
columnas = carga.get_cols_names_sin_tspan(diccionario = datos_sin_repetidos_anteriores)
diccionario_con_dfs_vacios = carga.crear_diccionario_con_dataframes_vacios(diccionario= datos_sin_repetidos_anteriores, keys=columnas)
datos_anteriores = carga.merge_df_vacio_con_datos(datos_sin_repetidos_anteriores, diccionario_con_dfs_vacios)
datos_anteriores = eliminar_datos_espurios(datos_anteriores)
datos_anteriores = carga.agregar_componentes_de_la_velocidad(datos_anteriores)

### Paso 4. Eliminar NaNs de los datos de las sondas.
1. Eliminar NaNs del inicio de los datos.
- si la sonda tiene mediciones anteriores a la fecha de estudio, se eliminan esos.
- si la sonda comienza a medir en el mes del estudio, se eliminan esos datos inciales.
2. Eliminar NaNs del final de los datos. (Solo se eliminan los datos del final del periodo de estudio)

In [ ]:
# Eliminar NaNs iniciales y finales del dataframe de cada sonda; almanecnar esa información en el excel de despliegue
datos_antes_del_estudio, datos_del_estudio = carga.eliminar_nans_iniciales(datos_anteriores, datos_finales)
datos_del_estudio = carga.eliminar_nans_finales(datos_del_estudio)

In [ ]:
# datos_anteriores["4878218"].head()

In [ ]:
# datos_antes_del_estudio["4878218"].head()

In [ ]:
carga.agregar_coordenadas_de_despliegue_corregidas_al_excel_de_despliegue(datos_antes_del_estudio, datos_del_estudio)

### Paso 5. Guardar datos procesados en formato pkl

In [ ]:
# Guardar datos anteriores a la fecha de estudio en archivo .pkl
carpeta_de_destino = crear_ruta_a_carpeta(get_carpeta_guardado_datos_procesados())
nombre_de_archivo = get_nombre_del_archivo_de_datos_previos_a_la_fecha_de_estudio()
guardar_diccionario_como_pickle(diccionario = datos_antes_del_estudio, 
                                ruta = carpeta_de_destino, 
                                nombre_archivo=nombre_de_archivo)

# Guardar datos del estudio en archivo .pkl
carpeta_de_destino = crear_ruta_a_carpeta(get_carpeta_guardado_datos_procesados())
nombre_de_archivo = get_nombre_archivo_datos_procesados()
guardar_diccionario_como_pickle(diccionario = datos_del_estudio, 
                                ruta = carpeta_de_destino, 
                                nombre_archivo=nombre_de_archivo)

In [ ]:
# datos_del_estudio["4878503"]